In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report as report
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier,RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
import matplotlib.pyplot as plt

In [5]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/cmc_train.csv', header = None)
data=df.values[1:]
data=data.astype(float)
X_train=data[:,0:9]
y_train=data[:,9]
y_train=y_train.astype(int)

df = pd.read_csv('data/cmc_test.csv', header = None)
data=df.values[1:]
data=data.astype(float)
X_test=data[:,0:9]
y_test=data[:,9]
y_test=y_test.astype(int)


# y_train = tf.one_hot(y_train,10)
# y_test = tf.one_hot(y_test,10)
# sess1=tf.Session()
# y_train, y_test = sess1.run([y_train,y_test])

In [30]:
lr_clf=LogisticRegression(max_iter=3000)
svm_clf=SVC()
rf_clf=RandomForestClassifier(
    n_estimators=21,
    criterion="gini",
    max_depth=4,
    random_state=0)

gbr = GradientBoostingClassifier(n_estimators=30000, max_depth=2, min_samples_split=2, learning_rate=0.01)
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=None, min_samples_split=20, min_samples_leaf=21),
                         algorithm="SAMME",
                         n_estimators=200, learning_rate=0.8)
xgb = XGBClassifier(learning_rate=0.01,max_depth=10)

In [ ]:
lr_clf.fit(X_train,y_train)
svm_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
gbr.fit(X_train, y_train.ravel())
bdt.fit(X_train,y_train)
eval_set = [(X_test, y_test)]
xgb.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="mlogloss", eval_set=eval_set, verbose=False)

In [32]:
lr_pred=lr_clf.predict(X_test)
svm_pred=svm_clf.predict(X_test)
rf_pred=rf_clf.predict(X_test)
gbr_pred = gbr.predict(X_test)
bdt_pred = bdt.predict(X_test)
xgb_pred = xgb.predict(X_test)

In [33]:
print('lr:\n',report(y_test,lr_pred))
print('svm:\n',report(y_test,svm_pred))
print('rf:\n',report(y_test, rf_pred))
print('GBDT:\n',report(y_test, gbr_pred))
print('AdaBoost:\n',report(y_test, bdt_pred))
print('XgBoost:\n',report(y_test, xgb_pred))

lr:
               precision    recall  f1-score   support

           1       0.57      0.55      0.56        84
           2       0.42      0.38      0.40        55
           3       0.44      0.49      0.46        82

    accuracy                           0.48       221
   macro avg       0.48      0.47      0.47       221
weighted avg       0.49      0.48      0.48       221

svm:
               precision    recall  f1-score   support

           1       0.47      0.38      0.42        84
           2       0.39      0.47      0.43        55
           3       0.45      0.48      0.46        82

    accuracy                           0.44       221
   macro avg       0.44      0.44      0.44       221
weighted avg       0.44      0.44      0.44       221

rf:
               precision    recall  f1-score   support

           1       0.66      0.45      0.54        84
           2       0.49      0.45      0.47        55
           3       0.46      0.63      0.54        82

    

In [34]:
esb_clf=VotingClassifier(estimators=[
    ('rf_clf',rf_clf),
    ('svm_clf',SVC(probability=True)),
    ('lr_clf',lr_clf),
    ('GBDT',gbr),
    ('AdaBoost',bdt),
    ('Xgboost',xgb)
    ],
    weights=[1,0.6,0.7,0.7,0.9,0.9],
    voting='soft')
esb_clf2=VotingClassifier(estimators=[
    ('rf_clf',rf_clf),
    ('svm_clf',SVC(probability=True)),
    ('lr_clf',lr_clf),
    ('GBDT',gbr),
    ('AdaBoost',bdt),
    ('Xgboost',xgb)],
    weights=[1,0.6,0.7,0.7,0.9,0.9],
    voting='hard')

In [35]:
esb_clf.fit(X_train,y_train)
esb_pred=esb_clf.predict(X_test)
esb_clf2.fit(X_train,y_train)
esb_pred2=esb_clf2.predict(X_test)

E:\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
E:\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", Fu

In [36]:
print('ensemble-soft:\n',report(y_test,esb_pred))
print('ensemble-hard:\n',report(y_test,esb_pred2))

ensemble-soft:
               precision    recall  f1-score   support

           1       0.58      0.58      0.58        84
           2       0.44      0.40      0.42        55
           3       0.47      0.50      0.49        82

    accuracy                           0.51       221
   macro avg       0.50      0.49      0.50       221
weighted avg       0.51      0.51      0.51       221

ensemble-hard:
               precision    recall  f1-score   support

           1       0.64      0.55      0.59        84
           2       0.45      0.45      0.45        55
           3       0.48      0.55      0.51        82

    accuracy                           0.52       221
   macro avg       0.52      0.52      0.52       221
weighted avg       0.53      0.52      0.53       221

